# CA1: Search Algorithms
## Amir Ali Vahidi 810199511



##### import needed libraries for further uses

In [155]:
from collections import deque
import copy
import time
import heapq
from colorama import Fore, Back, Style

##### Graph class for implementing the problem graph
parameters in __ __init__ __ function:
- `seyed_location`: current seyed location
- `edges`: graph edges
- `recipes`: two-dimensional list that contains every morid's needed recipes
- `morids`: list of locations of morids
- `difficult_vertices`: list of difficult vertices given in input
- `pass_count_difficult_vertices`: stores numbers of passes of difficult vertices(updated every time seyed passes a difficult vertice)
- `decreasing_count_dif_ver`: it is used for aknowledging of how long seyed should stay in a difficult vertice(initial value = pass_count_difficult_vertices and decreses when seyed is on a difficult vertice until it reaches 0 and then seyed is allowed to pass this difficult vertice)
- `path`: state path
- `total_cost`: cost of this state path
- `recipe_or_morid_visited_node_counted`: number of visited nodes in this path that contains either morid or recipe or both
- `not_empty_nodes`: number of all nodes in graph that contains either morid or recipe or both\

__ __iter__ __ functions is used to return a hash for a state based on three parameters:
- seyed current location
- morids visited and fully satisfied
- recipes gathered

In [156]:
class Graph:
    def __init__(self,seyed_location,edges,recipes,morids,difficult_vertices,pass_count_diffucult_vertices, decreasing_count_dif_ver,not_empty_nodes):
        self.seyed_location=seyed_location
        self.edges=edges
        self.recipes=recipes
        self.morids=morids
        self.difficult_vertices=difficult_vertices
        self.pass_count_diffucult_vertices=pass_count_diffucult_vertices
        self.decreasing_count_dif_ver= decreasing_count_dif_ver
        self.path=str(seyed_location)+" -> "
        self.total_cost=0
        self.recipe_or_morid_visited_node_counted=0
        self.not_empty_nodes=not_empty_nodes
    # def __hash__(self) -> int:
    #     print(tuple(self.seyed_location,tuple(self.pass_count_diffucult_vertices)))
    #     print(tuple(self.pass_count_diffucult_vertices))
    #     return hash((self.seyed_location,self.recipes,self.morids,self.pass_count_diffucult_vertices,self.decreasing_count_dif_ver))
    def __iter__(self):
        to_be_hashed =[]
        to_be_hashed.append(self.seyed_location)
        for morid in self.morids:           
            to_be_hashed.append(morid)
            for recipe in self.recipes[morid]:
                to_be_hashed.append(recipe)
        yield tuple(to_be_hashed)  

    def get_recipe_or_morids_visited_count(self):
        return self.recipe_or_morid_visited_node_counted
    
    def get_not_empty_nodes_count(self):
        return sum(self.not_empty_nodes) 
                      

In [91]:
def open_file(input_file_address):
    file = open(input_file_address)

    n, m = map(int,file.readline().split(" "))
    not_empty_nodes=[False for _ in range(n+1)]
    edges = []   
    for i in range(m):
        u, v = map(int,file.readline().split(" "))
        edges.append((u,v))
        
    h = int(file.readline())
    pass_count_dif_ver=[0 for _ in range(h)]
    decreasing_count_dif_ver=[0 for _ in range(h)]
    difficult_vertices=list(map(int,file.readline().split()))

    s= int(file.readline())
    morids_location=[]
    recipes=[[] for _ in range(n+1)]
    for _ in range(s):
        line = file.readline()
        morid_location=int(line.split()[0])
        morids_location.append(morid_location)
        not_empty_nodes[morid_location]=True
        morids_number_of_recipes=int(line.split()[1])
        recipes[morid_location]=list(map(int,line.split()[2:morids_number_of_recipes+2]))
        for recipe in recipes[morid_location]:
            not_empty_nodes[recipe]=True

    v = int(file.readline())

    return Graph(seyed_location=v,edges=edges,recipes=recipes,morids=morids_location,difficult_vertices=difficult_vertices,pass_count_diffucult_vertices=pass_count_dif_ver, decreasing_count_dif_ver= decreasing_count_dif_ver,not_empty_nodes=not_empty_nodes)

##### Update properties of the state and return a new state
parameters to check:
- `difficult vertices`: check if seyed current location is on difficult vertices and update decreasing_count_dif_ver and pass_count_diffucult_vertices if so
- `recipes`: if seyed current location contains a recipe it will be removed from wanted recipes list
- `morids`: if seyed current location contains a morid and morids wanted recipes are already gathered, this morid will be removed from wanted morids list

In [157]:
def expand(present_state: Graph,this_frontier):
    present_state = copy.deepcopy(present_state)
    present_state.seyed_location = this_frontier
    present_state.total_cost+=1
    # check if this_frontier is a difficult_vertices and update if so
    if this_frontier in present_state.difficult_vertices:
        index=present_state.difficult_vertices.index(this_frontier)
        # check if Seyed is thinking about purpose of Dizzy Mazgan
        if present_state.decreasing_count_dif_ver[index]>0:
            present_state.total_cost+=1
            present_state.decreasing_count_dif_ver[index]-=1
        # Seyed is not thinking about purpose of Dizzy Mazgan anymore
        else:
            present_state.pass_count_diffucult_vertices[index]+=1
            present_state.decreasing_count_dif_ver[index]=present_state.pass_count_diffucult_vertices[index]
    
    
    recipe_or_morid_visited=False

    # check if this_frontier contains a recipe
    for morid in present_state.morids: 
        if this_frontier in present_state.recipes[morid]:
            recipe_or_morid_visited=True
            present_state.recipes[morid].remove(this_frontier)
   
    # remove morids that have been visited
    if this_frontier in present_state.morids:
        # check if Morid can be visited(recipes of this Morid should be visited earlier)
        if len(present_state.recipes[this_frontier])==0:
            recipe_or_morid_visited=True
            present_state.morids.remove(this_frontier)
       
    #morids_to_be_remove_index=[index for (index, morid) in enumerate(present_state.morids) if morid == this_frontier]

    # count visited nodes containing morid or recipe
    if recipe_or_morid_visited:
        present_state.recipe_or_morid_visited_node_counted+=1

        
    return present_state

##### DZ Ready?
check if all `recipes` had beed gathered and all `morids` have recieved theirs and return "True" if so!

In [93]:
def is_final_state(state: Graph):
    number_of_morids=len(state.morids)
    if number_of_morids>0:
        return False
    for i in range(number_of_morids):
        if len(state.recipes[i])>0:
            return False
    return True

In [94]:
def state_encoder(state):
    return hash(tuple(state))

#### BFS (Uninformed)
starting of the initail seyed location, we add this state in the frontier_state queue.\
on every state, we first check whether it already exists in visited_states and if so, we continue the loop without calculating rest of this state path because it has already been calculated.
if this state has not already been visited, we add this state to the visited_states and check its adjacent nodes and if any of them satisfies the final state conditions, we save or update it's information and after finishing the serach of this depth of bfs, we return the rusult!\
after gaining this node's adjacent states, we add them to the frontier_state and continue the loop until a state satisfies the goal state conditions!

##### Pros:
- It return an optimal solution
- In Comparison to DFS and some other search algorithms, it has a better time complexity

##### Cons:
- In comparison to A* and some other search algorithms, it has a worse time complexity


In [95]:
def BFS(input_file_address):
    frontier_states=deque()
    visited_states=set()
    frontier_states.append(open_file(input_file_address))
    visited_state_count=1
    path,total_cost=0,float('inf')
    while(frontier_states):
        present_state=frontier_states.popleft()
        encoded_state=state_encoder(present_state)
        if encoded_state in visited_states:
            continue
        else:
            visited_states.add(encoded_state)
            visited_state_count+=1
        
        
        neighbors_dict=[item for item in present_state.edges if present_state.seyed_location in item]
        for neighbor in neighbors_dict:
            if neighbor[0]==present_state.seyed_location:
                this_frontier=neighbor[1]
            else:
                this_frontier=neighbor[0]

            new_state=expand(present_state=present_state,this_frontier=this_frontier)
            new_state.path+=str(new_state.seyed_location)

            if is_final_state(new_state):
                if total_cost > new_state.total_cost:
                    path,total_cost = new_state.path,new_state.total_cost

            new_state.path+=" -> "
            frontier_states.append(new_state)
    if path!=0:
        return(path,visited_state_count,total_cost)
    return (0,0,0)


##### BFS TEST

In [169]:
file_addresses = ['input.txt','input2.txt','input3.txt']
print(Back.BLUE+Fore.WHITE+'BFS TEST')
print(Style.RESET_ALL)
for file_address in file_addresses:
    time_sum=0
    for i in range(3):
        start_time=time.time()
        path, visited_states_number, cost=BFS(file_address)
        end_time=time.time()
        time_sum += end_time-start_time
    time_mean = time_sum/3
    print("BFS Result on "+Back.RED+ file_address+":")
    print(Style.RESET_ALL)
    print(Fore.GREEN+ "path: " +Fore.WHITE,path)
    print(Fore.GREEN+ "number of visited states:" +Fore.WHITE,str(visited_states_number))
    print(Fore.GREEN+ "minimum cost:" +Fore.WHITE ,str(cost))
    print(Fore.CYAN+"average time:"+Fore.WHITE,str(time_mean))
    print()
print(Style.RESET_ALL)

BFS TEST

BFS Result on input.txt:

path:  1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
number of visited states: 51
minimum cost: 8
average time: 0.009009361267089844

BFS Result on input2.txt:

path:  28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 23 -> 28 -> 23 -> 5 -> 7 -> 29
number of visited states: 3631
minimum cost: 12
average time: 5.56989057858785

BFS Result on input3.txt:

path:  40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
number of visited states: 3754
minimum cost: 21
average time: 2.983078718185425




Another test added by myself for testing difficult vertices because of a previously problem on a loop in graph

In [140]:
start_time=time.time()
path, visited_states_number,cost=BFS("input4.txt")
end_time=time.time()
print("test for difficult vertices problem:")
print(path,visited_states_number,cost)
print(end_time-start_time)

test for difficult vertices problem:
1 -> 2 -> 3 -> 2 -> 1 -> 10 29 6
0.0035517215728759766


#### IDS
In IDS function, starting with depth 1, we increment the max_depth for algorithm to find the result recursively using dfs and if a state satisfies the condition of the final state, it returns the result and if not and max_depth is reached, it returns "0" as a path(0,visited_state_count,0) and we continue to search for the result in a deeper depth.

##### Pros:
- It return the optimal solution
- It requires lesser memory compare to BFS and A*
##### Cons:
- It runs slow due to possiblity of visiting some states more than once



In [170]:
# Depth-Limited search
def DLS(initial_state,visited_states,depth,max_depth):
    
    visited_states=copy.deepcopy(visited_states)
    frontier_states = deque()
    frontier_states.append(initial_state)
    visited_state_count = 1
    path,total_cost=0,float('inf')
    while (frontier_states):
        present_state = frontier_states.pop()
        encoded_state = state_encoder(present_state)
        if encoded_state in visited_states:
            if visited_states[encoded_state]<=depth:
                continue
        else:
            visited_states[encoded_state]=depth
            visited_state_count += 1      
        
        if is_final_state(present_state):   
            return (present_state.path, visited_state_count, present_state.total_cost)

        if depth+1 > max_depth:
            return (0,visited_state_count,0)

        neighbors_dict=[item for item in present_state.edges if present_state.seyed_location in item]
        for neighbor in neighbors_dict:
            if neighbor[0]==present_state.seyed_location:
                this_frontier=neighbor[1]
            else:
                this_frontier=neighbor[0]
            
            new_state = expand(present_state=present_state,this_frontier=this_frontier)
            new_state.path += str(new_state.seyed_location)
            new_state.path += " -> "
            
            path1 , total_visited_state_count , total_cost1 = DLS(initial_state= new_state,visited_states=visited_states,depth=depth+1,max_depth=max_depth)
            visited_state_count+=total_visited_state_count         
            
            # check if there is a better way in this depth
            if path1 != 0:
                if total_cost > total_cost1:
                    path,total_cost = path1,total_cost1
    
    if path!=0:
        return(path,visited_state_count,total_cost)
    return (0,visited_state_count,0)
    


def IDS(input_file_address):
    
    initial_state=open_file(input_file_address)
    depth=0
    total_visited_state_number=0
    
    while True:
        visited_states={}
        path, visited_states_number, cost=DLS(initial_state=initial_state,visited_states=visited_states,depth=0,max_depth=depth)
        total_visited_state_number+=visited_states_number
        if path!=0:
            return (path, total_visited_state_number, cost,depth) 
        depth += 1

#### IDS TEST

In [175]:
file_addresses = ['input-easy.txt','input2-easy.txt','input3-easy.txt']
print(Back.BLUE+Fore.WHITE+'IDS TEST')
print(Style.RESET_ALL)
for file_address in file_addresses:
    time_sum=0
    for i in range(3):
        start_time=time.time()
        path, visited_states_number, cost,depth=IDS(file_address)
        end_time=time.time()
        time_sum += end_time-start_time
    time_mean = time_sum/3
    print("IDS Result on "+Back.RED+ file_address+":"+Back.RESET)
    print(Fore.GREEN+ "path: " +Fore.WHITE,path)
    print(Fore.GREEN+ "number of visited states:" +Fore.WHITE,str(visited_states_number))
    print(Fore.GREEN+ "minimum cost:" +Fore.WHITE ,str(cost))
    print(Fore.GREEN+ "depth reached:" +Fore.WHITE,str(depth))
    print(Fore.CYAN+"average time:"+Fore.WHITE,str(time_mean))
    print()
    print(Style.RESET_ALL)

IDS TEST

IDS Result on input-easy.txt:
path:  1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8 -> 
number of visited states: 1860
minimum cost: 8
depth reached: 8
average time: 0.0762181282043457


IDS Result on input2-easy.txt:
path:  9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8 -> 
number of visited states: 3639
minimum cost: 8
depth reached: 8
average time: 0.15729951858520508


IDS Result on input3-easy.txt:
path:  13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10 -> 
number of visited states: 75705
minimum cost: 13
depth reached: 13
average time: 3.320495367050171




This heap is specialized for performing push and pop based on the key given to it.\
Key = (total cost of the path) + (alpha * defined heuristic) 

In [101]:
class StateHeap:
    def __init__(self, key=lambda x:x):
        self.key = key
        self.index = 0
        self._data = []

    def push(self, item):
        heapq.heappush(self._data, (self.key(item), self.index, item))
        self.index += 1

    def pop(self):
        return heapq.heappop(self._data)[2]

    def __bool__(self):
        return bool(self._data)

##### Heuristic for A*
we choose our heuristic number of the nodes that has morids or recipes and they have not been visited yet.\
for a heuristic function to be consistent, we must prove for every successor $P$ of $N$ :
$$h(N)\leq c(N,P)+h(P)$$
In every new state a Morid may be satisfy or a new recipe may be collected. Therefore for every successor $P$ of $N$ we have $h(N) \leq h(P)$. Between every two nodes in our graph we have $c(U,V)\geq 0$.\
So in this heuristic, $h(N)\leq c(N,P)+h(P)$ and therefor heuristic is consistent!

In [142]:
def heuristic(state : Graph):
    not_empty_nodes= state.get_not_empty_nodes_count()
    visited_morid_recipe_count = state.get_recipe_or_morids_visited_count()
    to_be_visited_nodes_count = not_empty_nodes-visited_morid_recipe_count
    return to_be_visited_nodes_count

#### A*
It is a mix of BFS and heuristic function.
##### Pros:
- It returns an optimal solution
- It has a very good time complexity due to visiting less states in comparison with IDS and BFS
##### Cons:
- Sometimes it's hard to find a good heuristic

In [143]:
def A_star(input_file_address,alpha=1):

    frontier_states=StateHeap(lambda state: state.total_cost + alpha*heuristic(state))
    visited_states=set()
    initial_state=open_file(input_file_address)
    frontier_states.push(initial_state)
    
    visited_state_count=1
    path,total_cost=0,float('inf')
    while(frontier_states):
        present_state=frontier_states.pop()
        encoded_state=state_encoder(present_state)
        if encoded_state in visited_states:
            continue
        else:
            visited_states.add(encoded_state)
            visited_state_count+=1
        
        
        neighbors_dict=[item for item in present_state.edges if present_state.seyed_location in item]
        for neighbor in neighbors_dict:
            if neighbor[0]==present_state.seyed_location:
                this_frontier=neighbor[1]
            else:
                this_frontier=neighbor[0]

            new_state=expand(present_state=present_state,this_frontier=this_frontier)
            new_state.path+=str(new_state.seyed_location)

            if is_final_state(new_state):
                if total_cost > new_state.total_cost:
                    path,total_cost = new_state.path,new_state.total_cost

            new_state.path+=" -> "
            frontier_states.push(new_state)
           
            if path!=0:
                return(path,visited_state_count,total_cost)
    return (0,0,0)


A* TEST ($\alpha=1$)

In [176]:
# A* test on alpha=1
file_addresses = ['input.txt','input2.txt','input3.txt']
print(Back.BLUE+Fore.WHITE+'A* TEST(alpha=1)')
print(Style.RESET_ALL)
for file_address in file_addresses:
    time_sum=0
    for i in range(3):
        start_time=time.time()
        path, visited_states_number, cost=A_star(file_address)
        end_time=time.time()
        time_sum += end_time-start_time
        #print(path, visited_states_number, cost,depth)
    time_mean = time_sum/3
    print("A* Result on "+Back.RED+ file_address+":")
    print(Style.RESET_ALL)
    print(Fore.GREEN+ "path: " +Fore.WHITE,path)
    print(Fore.GREEN+ "number of visited states:" +Fore.WHITE,str(visited_states_number))
    print(Fore.GREEN+ "minimum cost:" +Fore.WHITE ,str(cost))
    print(Fore.CYAN+"average time:"+Fore.WHITE,str(time_mean))
    print()
print(Style.RESET_ALL)

A* TEST(alpha=1)

A* Result on input.txt:

path:  1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
number of visited states: 29
minimum cost: 8
average time: 0.005620241165161133

A* Result on input2.txt:

path:  28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 23 -> 5 -> 7 -> 29 -> 22 -> 28
number of visited states: 1046
minimum cost: 12
average time: 1.7522508303324382

A* Result on input3.txt:

path:  40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
number of visited states: 2827
minimum cost: 21
average time: 2.151686429977417




#### A* Weighted
##### Pros:
- visits lesser states than normal A* and better time complexity
##### Cons:
- if $\alpha$ is not choosen carefully, it's answer may not be optimal.(like $\alpha=5$ in this example for test3)

A* TEST ($\alpha=1.4$)

In [177]:
# A* test on alpha=1.4
file_addresses = ['input.txt','input2.txt','input3.txt']
file_addresses = ['input.txt','input2.txt','input3.txt']
print(Back.BLUE+Fore.WHITE+'A* TEST(alpha=1.4)')
print(Style.RESET_ALL)
for file_address in file_addresses:
    time_sum=0
    for i in range(3):
        start_time=time.time()
        path, visited_states_number, cost=A_star(file_address,1.4)
        end_time=time.time()
        time_sum += end_time-start_time
        #print(path, visited_states_number, cost,depth)
    time_mean = time_sum/3
    print("A* Result on "+Back.RED+ file_address+":")
    print(Style.RESET_ALL)
    print(Fore.GREEN+ "path: " +Fore.WHITE,path)
    print(Fore.GREEN+ "number of visited states:" +Fore.WHITE,str(visited_states_number))
    print(Fore.GREEN+ "minimum cost:" +Fore.WHITE ,str(cost))
    print(Fore.CYAN+"average time:"+Fore.WHITE,str(time_mean))
    print()
print(Style.RESET_ALL)

A* TEST(alpha=1.4)

A* Result on input.txt:

path:  1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
number of visited states: 24
minimum cost: 8
average time: 0.0042870839436848955

A* Result on input2.txt:

path:  28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 23 -> 5 -> 7 -> 29 -> 22 -> 28
number of visited states: 45
minimum cost: 12
average time: 0.10704700152079265

A* Result on input3.txt:

path:  40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
number of visited states: 2225
minimum cost: 21
average time: 1.7981499830881755




A* TEST ($\alpha=5$)

In [178]:
# A* test on alpha=5
file_addresses = ['input.txt','input2.txt','input3.txt']
print(Back.BLUE+Fore.WHITE+'A* TEST(alpha=5)')
print(Style.RESET_ALL)
for file_address in file_addresses:
    time_sum=0
    for i in range(3):
        start_time=time.time()
        path, visited_states_number, cost=A_star(file_address,5)
        end_time=time.time()
        time_sum += end_time-start_time
        #print(path, visited_states_number, cost,depth)
    time_mean = time_sum/3
    print("A* Result on "+Back.RED+ file_address+":")
    print(Style.RESET_ALL)
    print(Fore.GREEN+ "path: " +Fore.WHITE,path)
    print(Fore.GREEN+ "number of visited states:" +Fore.WHITE,str(visited_states_number))
    print(Fore.GREEN+ "minimum cost:" +Fore.WHITE ,str(cost))
    print(Fore.CYAN+"average time:"+Fore.WHITE,str(time_mean))
    print()
print(Style.RESET_ALL)

A* TEST(alpha=5)

A* Result on input.txt:

path:  1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
number of visited states: 15
minimum cost: 8
average time: 0.0028246243794759116

A* Result on input2.txt:

path:  28 -> 19 -> 3 -> 11 -> 24 -> 9 -> 23 -> 5 -> 7 -> 29 -> 20 -> 13 -> 19 -> 28
number of visited states: 16
minimum cost: 13
average time: 0.02593088150024414

A* Result on input3.txt:

path:  40 -> 42 -> 38 -> 24 -> 31 -> 15 -> 12 -> 16 -> 12 -> 49 -> 9 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 10 -> 41 -> 48 -> 41 -> 18 -> 1 -> 19
number of visited states: 76
minimum cost: 25
average time: 0.08561595280965169




### `OVERALL RESULT`

##### **input.txt**

| |Minimum Cost|States Count|Average Time|Depth Reached|
|:---|:----:|:----:|:----:|:----:|
|BFS|8|51|0.0090|-|
|IDS|8|1860|0.0762|8|
|A*|8|29|0.0056|-|
|Weighted A* 1.4|8|24|0.0042|-|
|Weighted A* 5|8|15|0.0028|-|

##### **input2.txt** (input2-easy.txt for IDS)

| |Minimum Cost|States Count|Average Time|Depth Reached|
|:---|:----:|:----:|:----:|:----:|
|BFS|12|3631|5.5698|-|
|IDS|8|3639|0.1572|8|
|A*|12|1046|1.7522|-|
|Weighted A* 1.4|12|45|0.1070|-|
|Weighted A* 5|13|16|0.0259|-|

##### **input3.txt** (input3-easy.txt for IDS)

| |Minimum Cost|States Count|Average Time|Depth Reached|
|:---|:----:|:----:|:----:|:----:|
|BFS|21|3754|2.9830|-|
|IDS|13|75705|3.3204|13|
|A*|21|2827|2.1516|-|
|Weighted A* 1.4|21|2225|1.7981|-|
|Weighted A* 5|25|76|0.0856|-|